In [26]:
import itertools
import math
import pickle
import random
import sys
import time
from multiprocessing import Pool

import geopy
import networkx as nx
import numpy as np
import pandas
import pandas as pd
from scipy.spatial import KDTree as kd
from tqdm import trange, tqdm
# import matplotlib.pyplot as plt
import graph_tests
import graph_generator
import clustering
import osmnx as ox
import time
from common import GraphLayer
import numpy as np
import matplotlib.pyplot as plt
import pylab

In [27]:
G = graph_generator.get_graph('R2555133')

In [58]:
Q = nx.Graph()
for u, d in G.nodes(data=True):
    Q.add_node(u, x=d['x'], y=d['y'])
for u, v, d in G.edges(data=True):
    if u != v:
        Q.add_edge(u, v, length=d['length'])
G = Q

In [29]:
# print(G.edges(data=True))

In [59]:
communities = clustering.LouvainKMeansCommunitiesResolver(resolution = 50).find_communities(G)
clustering.CommunitiesResolver.write_communities(G, communities)
cls2hubs = graph_generator.get_cluster_to_bridge_points(G)
cls2n = graph_generator.get_cluster_to_neighboring_clusters(G)

r: 50 length


In [60]:
print(len(communities))

1264


In [61]:
c_from = 0
c_to = 1

nodes_from = communities[c_from]
nodes_to = communities[c_to]


In [62]:
a = list(dict.fromkeys([1,444,555,666]))
b = list(dict.fromkeys([666,555,444,1]))
print(a,b)

[1, 444, 555, 666] [666, 555, 444, 1]


In [63]:
c_from = 0
c_to = 1

nodes_from = communities[c_from]
nodes_to = communities[c_to]

statistic = {}
total = 0
for f in tqdm(nodes_from):
    for t in nodes_to:
        p = nx.single_source_dijkstra(G, f,t,weight='length')
        cls = list(dict.fromkeys([int(G.nodes[u]['cluster']) for u in p[1]])) 
        cls = tuple(cls)
        if cls not in statistic:
            statistic[cls] = 0
        statistic[cls]+=1
        total +=1
for k in statistic:
    statistic[k]/=total


100%|██████████| 12/12 [00:00<00:00, 5826.77it/s]


In [64]:
statistic

{(0, 3, 1): 1.0}

In [36]:
f = {str(k): statistic[k] for k in statistic}
f = list(f.items())
f.sort(key = lambda x : -x[1])
f = dict(f)
f

{'(0, 1)': 0.9166666666666666,
 '(0, 2, 9, 1)': 0.06111111111111111,
 '(0, 57, 1)': 0.022222222222222223}

In [65]:
def get_statistic(c_from,c_to, log = False):
    nodes_from = communities[c_from]
    nodes_to = communities[c_to]
    
    statistic = {}
    total = 0
    generator = tqdm(nodes_from, desc='find statistic') if log else nodes_from 
    for f in generator:
        for t in nodes_to:
            p = nx.single_source_dijkstra(G, f,t,weight='length')
            cls = list(dict.fromkeys([int(G.nodes[u]['cluster']) for u in p[1]])) 
            
            cls = tuple(cls)
            if cls not in statistic:
                statistic[cls] = 0
            statistic[cls]+=1
            total +=1
    for k in statistic:
        statistic[k]/=total
    statistic = list(statistic.items())
    statistic.sort(key = lambda x : -x[1])
    statistic = dict(statistic)
    return statistic

In [66]:
def get_statistic_per_point(c_from,c_to,cls2hubs, log = False):
    # nodes_from =[u for u in nx.barycenter(extract_cluster_list_subgraph(G, [c_from], communities), weight='length')]
    # nodes_to = [u for u in  nx.barycenter(extract_cluster_list_subgraph(G, [c_to], communities), weight='length')]
    nodes_from = cls2hubs[c_from]
    nodes_to = cls2hubs[c_to]
    statistic = {}
    generator = tqdm(nodes_from, desc='find statistic') if log else nodes_from 
    # generator = [nx.barycenter(extract_cluster_list_subgraph(G, [c_from], communities))[0]]
    for f in generator:
        statistic[f] = {}
        for t in nodes_to:
            p = nx.single_source_dijkstra(G, f,t,weight='length')
            cls = list(dict.fromkeys([int(G.nodes[u]['cluster']) for u in p[1]])) 
            cls = tuple(cls)
            if cls not in statistic[f]:
                statistic[f][cls] = 0
            statistic[f][cls] += 1
    return statistic

In [39]:
print(get_statistic(0,0, cls2hubs))

find statistic: 100%|██████████| 18/18 [00:00<00:00, 2023.68it/s]

{(0,): 0.9320987654320988, (0, 1): 0.05555555555555555, (0, 2): 0.012345679012345678}


In [67]:
def get_distance(c_from,c_to, c = None,log = False):
    nodes_from = communities[c_from]
    nodes_to = communities[c_to]
    q = G if c is None else graph_generator.extract_cluster_list_subgraph(G, c, communities)
    dist = {}
    add = ' usual' if c is None else ' with clusters'
    generator =  tqdm(nodes_from, desc='find paths' + add) if log else nodes_from
    for f in generator:
        for t in nodes_to:
            p = nx.single_source_dijkstra(q, f,t,weight='length')
            if f not in dist:
                dist[f] = {}
            dist[f][t] = p[0]
    return dist

In [41]:
usual_matrix =  get_distance(0,1)

In [42]:
c1 = set([0, 268, 437, 27, 74, 385, 355, 4, 109, 311, 105, 3, 384, 427, 417, 348, 39, 18, 98, 99, 143, 144, 148, 149, 150, 451, 395, 153, 1])
c2 = set([0, 369, 268, 331, 239, 355, 4, 109, 311, 105, 3, 384, 427, 417, 348, 39, 18, 98, 99, 143, 144, 148, 149, 150, 451, 395, 153, 1])
cc = c1.union(c2)
cluster_matrix =  get_distance(0,1, cc)

In [43]:
err = {f : {t : abs(usual_matrix[f][t] - cluster_matrix[f][t])/usual_matrix[f][t] for t in usual_matrix[f]}for f in usual_matrix}

In [44]:
errs = [err[f][t] for f in err for t in err[f]]

In [45]:
print(np.mean(errs))

0.009963765379895963


In [47]:
import time

data = {}
top_clusters = 1
for i in range(len(communities)):
    data[i] = {}
    for j in trange(i+1, len(communities)):
        st = list(get_statistic(i,j).items())
        c = set()
        for cls in range(top_clusters):      
            c1 = set(st[cls][0])
            c = c.union(c1)
        start = time.time_ns()
        usual_matrix =  get_distance(i,j)
        usual_time = time.time_ns() - start
        start = time.time_ns()
        cluster_matrix =  get_distance(i,j, c)
        cluster_time = time.time_ns() - start
        acc = usual_time / cluster_time
        err = {f : {t : abs(usual_matrix[f][t] - cluster_matrix[f][t])/usual_matrix[f][t] for t in usual_matrix[f]}for f in usual_matrix}
        errs = [err[f][t] for f in err for t in err[f]]
        print(f'from {i} to {j} mean: {np.mean(errs):3f}±{np.std(errs):3f} max: {np.max(errs):3f} acceleration: {acc:3f}')
        data[i][j] = errs

  1%|          | 4/790 [00:00<00:50, 15.47it/s]

from 0 to 1 mean: 0.009964±0.044952 max: 0.412521 acceleration: 1.515556
from 0 to 2 mean: 0.000237±0.001822 max: 0.016676 acceleration: 1.389875
from 0 to 3 mean: 0.009033±0.034654 max: 0.230514 acceleration: 1.014588
from 0 to 4 mean: 0.035176±0.090612 max: 0.562528 acceleration: 1.267880


  1%|          | 6/790 [00:00<01:13, 10.70it/s]

from 0 to 5 mean: 0.008364±0.047737 max: 0.506380 acceleration: 1.473704
from 0 to 6 mean: 0.002773±0.010250 max: 0.054738 acceleration: 2.220538


  1%|          | 9/790 [00:00<00:59, 13.04it/s]

...

  2%|▏         | 16/790 [00:12<14:48,  1.15s/it]

from 0 to 16 mean: 0.018934±0.025125 max: 0.071241 acceleration: 5.019406


  2%|▏         | 17/790 [00:13<15:51,  1.23s/it]

from 0 to 17 mean: 0.039027±0.036149 max: 0.155441 acceleration: 5.526257


  2%|▏         | 17/790 [00:15<12:04,  1.07it/s]

KeyboardInterrupt



In [48]:
def extract_cluster_list_subgraph(graph: nx.Graph, cluster_number: list[int] | set[int], communities=None) -> nx.Graph:
    if communities:
        nodes_to_keep = [u for c in cluster_number for u in communities[c]]
    else:
        nodes_to_keep = [node for node, data in graph.nodes(data=True) if data['cluster'] in cluster_number]
    return  nx.Graph(graph.subgraph(nodes_to_keep))

In [ ]:
def draw():
    pass

In [68]:
A = {}
for i in trange(0,len(communities)):
    A[i] = {}
    for j in cls2n[i]:
        data = get_statistic_per_point(i,j,cls2hubs,False)
        ss= set()
        for k,v in data.items():
            for kk, vv in v.items():
                ss.add(kk)
        cls0to1 = set()
        for ii in ss:
            for jj in ii:
                cls0to1.add(jj)
        A[i][j] = cls0to1

100%|██████████| 1264/1264 [00:46<00:00, 27.00it/s] 


print

In [103]:
c_diff = 0
c_eq = 0
for i in A:
    for j in A[i]:
        a : set = cls2n[i]
        a = a.union(set([i])).union(cls2n[j])
        b : set = A[i][j]
        delta = b.difference(a)
        if len(delta) != 0:
            c_diff+=1
        else:
            c_eq+=1
print(c_diff, c_eq)

264 5106


In [69]:
count_data = {}
for i in A:
    for j in A[i]:
        l = len(A[i][j])
        if l not in count_data:
            count_data[l] = 0
        count_data[l]+=1
        # if len(A[i][j]) > 2:
            # print(i,j,A[i][j])
print(count_data)

{2: 1510, 5: 694, 4: 1128, 3: 1504, 6: 270, 7: 144, 8: 74, 10: 10, 9: 26, 13: 6, 14: 2, 11: 2}


r: 50 length


100%|██████████| 205/205 [00:00<00:00, 1370.48it/s]


,n,count
0,5,564
1,3,1028
2,4,848
3,6,282
4,2,798
5,8,30
6,7,110
7,9,18
8,10,6


In [72]:
data = {'n' : [k for k in count_data],
        'count': [count_data[k]/2 for k in count_data]}
df = pandas.DataFrame.from_dict(data)

In [73]:
df

,n,count
0,2,755.0
1,5,347.0
2,4,564.0
3,3,752.0
4,6,135.0
5,7,72.0
6,8,37.0
7,10,5.0
8,9,13.0
9,13,3.0


In [84]:
layer = graph_generator.generate_layer(G,0, communities = communities,paths = 'first')
layer = layer[0]

In [85]:
for i,c in enumerate(communities):
    print(len(c)/len(cls2hubs[i]))

1.7142857142857142
1.5
2.0
2.0
1.5
1.6666666666666667
1.6
2.2
1.0
1.5
9.0
1.8333333333333333
2.090909090909091
3.2
3.6
1.5714285714285714
1.5454545454545454
2.625
2.5
2.625
4.666666666666667
1.8
1.75
2.1
2.7142857142857144
2.1666666666666665
1.8181818181818181
2.1
1.1818181818181819
3.5
1.6153846153846154
2.7142857142857144
2.625
2.0384615384615383
2.25
2.5
3.0
1.75
1.0
2.0
3.3333333333333335
1.5
1.6666666666666667
1.8333333333333333
1.5
2.0
3.0
2.8
2.9
1.125
1.8461538461538463
1.0
2.0833333333333335
2.875
1.25
3.5
1.0
3.125
3.2
3.0
1.875
2.3
1.3
1.6666666666666667
1.25
4.25
1.8
1.8333333333333333
1.0
1.4615384615384615
2.0
2.0
3.2222222222222223
1.875
1.125
1.0
1.0
1.6363636363636365
1.8571428571428572
2.1666666666666665
1.6666666666666667
6.4
1.8571428571428572
2.1666666666666665
2.0
2.0
2.0
1.5454545454545454
4.0
2.0
1.5454545454545454
1.7777777777777777
1.1428571428571428
2.142857142857143
1.0
1.0
2.4
2.25
2.6666666666666665
2.5
2.8
3.0
1.8666666666666667
1.125
1.5789473684210527
2

In [86]:
f1 = 0
f2 = 72
nodes_from = communities[f1]
nodes_to = communities[f2]

u1 = layer.cluster_to_center[f1]
u2 = layer.cluster_to_center[f2]
p = nx.single_source_dijkstra(layer.centroids_graph, u1,u2,weight='length')
clusters = set()
for i in range(0,len(p[1])-1):
    c1 = G.nodes[p[1][i]]['cluster']
    c2 = G.nodes[p[1][i+1]]['cluster']
    if c1 in A:
        cc = A[c1][c2]
    else:
        cc = A[c2][c1]
    for ccc in cc:
        clusters.add(ccc)

In [87]:
u1 = layer.cluster_to_center[f1]
u2 = layer.cluster_to_center[f2]
p = nx.single_source_dijkstra(layer.centroids_graph, u1,u2,weight='length')
clusters = set()
for i in range(0,len(p[1])-1):
    c1 = G.nodes[p[1][i]]['cluster']
    c2 = G.nodes[p[1][i+1]]['cluster']
    if c1 in A:
        cc = A[c1][c2]
    else:
        cc = A[c2][c1]
    for ccc in cc:
        clusters.add(ccc)
        
for uu1 in cls2n[f1]:
    for uu2 in cls2n[f2]:
        u1 = layer.cluster_to_center[uu1]
        u2 = layer.cluster_to_center[uu2]
        p = nx.single_source_dijkstra(layer.centroids_graph, u1,u2,weight='length')
        clusters = set()
        for i in range(0,len(p[1])-1):
            c1 = G.nodes[p[1][i]]['cluster']
            c2 = G.nodes[p[1][i+1]]['cluster']
            if c1 in A:
                cc = A[c1][c2]
            else:
                cc = A[c2][c1]
            for ccc in cc:
                clusters.add(ccc)
        

In [88]:
cls0to1 = clusters      
print(len(cls0to1))
GG = extract_cluster_list_subgraph(G, cls0to1, communities)
print(len(G.nodes), len(GG.nodes))
print(len(G.edges), len(GG.edges))

31
17921 465
27013 647


In [89]:
f1 = 0
f2 = 72
nodes_from = communities[f1]
nodes_to = communities[f2]

paths = set()
pp = {}
for f in nodes_from:
    for to in nodes_to:
        p = nx.dijkstra_path(G, f, to, weight='length')
        cls = tuple(list(dict.fromkeys([G.nodes[u]['cluster'] for u in p])))
        if cls not in pp:
            pp[cls] = 0
        pp[cls]+=1
        paths.add(cls)
for i in pp:
    print(i, pp[i])

(0, 6, 3, 7, 236, 927, 921, 919, 924, 88, 85, 81, 72) 28
(0, 6, 3, 7, 236, 927, 921, 919, 924, 87, 80, 85, 82, 83, 64, 72) 1
(0, 3, 7, 236, 927, 921, 919, 924, 88, 85, 81, 72) 224
(0, 3, 7, 236, 927, 921, 919, 924, 87, 80, 85, 82, 83, 64, 72) 8
(0, 4, 3, 7, 236, 927, 921, 919, 924, 88, 85, 81, 72) 84
(0, 4, 3, 7, 236, 927, 921, 919, 924, 87, 80, 85, 82, 83, 64, 72) 3


In [80]:
paths = set([vv for uu in paths for vv in uu ])
print(paths)

{0, 3, 4, 6, 7, 919, 921, 924, 927, 64, 72, 80, 81, 82, 83, 85, 87, 88, 236}


In [81]:
stat1 = {}
stat2 = {}



start = time.time_ns()
for f in nodes_from:
    stat1[f] ={}
    for to in nodes_to:
        p = nx.shortest_path_length(G, f, to, weight='length')
        stat1[f][to] = p
delta1 = time.time_ns() - start
print(f'usual time: {delta1/1000000:.2f}')

start = time.time_ns()
for f in nodes_from:
    stat2[f] ={}
    for to in nodes_to:
        p = nx.shortest_path_length(GG, f, to, weight='length')
        stat2[f][to] = p
delta2 = time.time_ns() - start
print(f'cls time: {delta2/1000000:.2f}')
errs = []
errs_abs = []
for i in nodes_from:
    for j in nodes_to:
        errs.append(abs(stat1[i][j]- stat2[i][j])/stat1[i][j])
        errs_abs.append(abs(stat1[i][j]- stat2[i][j]))
print(f'acceleration: {delta1/delta2:.2f}')
print(f'mean err {np.mean(errs):.3f}%; max err{ np.max(errs):.3f}%')
print(f'mean err {np.mean(errs_abs):.3f}; max err{ np.max(errs_abs):.3f}')
print('mean path len: ',np.mean([stat1[i][j] for i in nodes_from for j in nodes_to]))

# 
# start = time.time_ns()
# for f in nodes_from:
#     stat2[f] ={}
#     for to in nodes_to:
#         p = nx.astar_path_length(GG, f, to, weight='length', heuristic=h)
#         stat2[f][to] = p
# delta = time.time_ns() - start
# print(f'cls time a star: {delta/1000000:.2f}')
# errs = []
# errs_abs = []
# for i in nodes_from:
#     for j in nodes_to:
#         errs.append(abs(stat1[i][j]- stat2[i][j])/stat1[i][j])
#         errs_abs.append(abs(stat1[i][j]- stat2[i][j]))
# print(f'mean err {np.mean(errs):.3f}%; max err{ np.max(errs):.3f}%')
# print(f'mean err {np.mean(errs_abs):.3f}; max err{ np.max(errs_abs):.3f}')
# print('mean path len: ',np.mean([stat1[i][j] for i in nodes_from for j in nodes_to]))

usual time: 3153.55


NodeNotFound: Node 239532064 not found in graph

In [ ]:
e = np.array(errs)
e = e[e>0]
plt.hist(e)

In [ ]:
sub_graph_all = extract_cluster_list_subgraph(G, paths, communities)
pos_all= { u : (d['x'],d['y']) for u,d in sub_graph_all.nodes(data=True)}

sub_graph = extract_cluster_list_subgraph(G, {f1,f2}, communities)
pos= { u : (d['x'],d['y']) for u,d in sub_graph.nodes(data=True)}
# nx.draw_networkx_edge_labels(Q,pos)
# nx.draw(G,pos, node_color =    values, node_size=1500,edge_color=edge_colors,edge_cmap=plt.cm.Reds)
plt.figure(figsize=(10,10))
nx.draw(sub_graph_all,pos_all, node_size=20,edge_cmap=plt.cm.Reds, alpha = 0.5)
nx.draw(sub_graph,pos,node_color = 'red', node_size=20,edge_cmap=plt.cm.Reds)

pylab.show()

In [82]:
max_err = 0
max_err_h_path = None
max_err_path = None

for f in nodes_from:
    for to in nodes_to:
        p = nx.single_source_dijkstra(G, f, to, weight='length')
        p_h = nx.single_source_dijkstra(GG, f, to, weight='length')
        err = abs(p[0] - p_h[0]) / p[0]
        if err > max_err:
            max_err = err
            max_err_path = p[1]
            max_err_h_path = p_h[1]

NodeNotFound: Node 239532064 not found in graph

In [ ]:
print(max_err)

In [ ]:
cluusters_usual = set([G.nodes[u]['cluster'] for u in max_err_path])
cluusters_h = set([G.nodes[u]['cluster'] for u in max_err_h_path])

sub_graph_all = extract_cluster_list_subgraph(G, cluusters_usual, communities)
pos_all= { u : (d['x'],d['y']) for u,d in sub_graph_all.nodes(data=True)}

sub_graph = extract_cluster_list_subgraph(G, cluusters_h, communities)
pos= { u : (d['x'],d['y']) for u,d in sub_graph.nodes(data=True)}

sub_graph_path = nx.subgraph(G, max_err_path)
pos_sub_graph_path= { u : (d['x'],d['y']) for u,d in sub_graph_path.nodes(data=True)}


# nx.draw_networkx_edge_labels(Q,pos)
# nx.draw(G,pos, node_color =    values, node_size=1500,edge_color=edge_colors,edge_cmap=plt.cm.Reds)
plt.figure(figsize=(10,10))
# nx.draw(sub_graph_all,pos_all, node_size=20,edge_cmap=plt.cm.Reds, alpha = 0.5)
# nx.draw(sub_graph,pos,node_color = 'red', node_size=20,edge_cmap=plt.cm.Reds)
nx.draw(sub_graph_path,pos_sub_graph_path,node_color = 'green', node_size=20,edge_cmap=plt.cm.Reds, alpha = 0.5)
nx.draw(GG,{ u : (d['x'],d['y']) for u,d in GG.nodes(data=True)},node_color = 'gray', node_size=20,edge_cmap=plt.cm.Reds, alpha = 0.5)

pylab.show()

In [ ]:
cls0to1 = set()
for i in ss:
    for j in i:
        cls0to1.add(j)

In [ ]:
print(cls0to1)

In [ ]:
GG = extract_cluster_list_subgraph(G, cls0to1, communities)
print(len(G.nodes), len(GG.nodes))
print(len(G.edges), len(GG.edges))

In [ ]:
f1 = 0
f2 = 10

stat1 = {}
stat2 = {}

nodes_from = communities[f1]
nodes_to = communities[f2]

start = time.time_ns()
for f in nodes_from:
    stat2[f] ={}
    for to in nodes_to:
        p = nx.shortest_path_length(GG, f, to, weight='length')
        stat2[f][to] = p
delta = time.time_ns() - start
print(f'cls time: {delta/1000000:.2f}')
start = time.time_ns()
for f in nodes_from:
    stat1[f] ={}
    for to in nodes_to:
        p = nx.shortest_path_length(G, f, to, weight='length')
        stat1[f][to] = p
delta = time.time_ns() - start
print(f'usual time: {delta/1000000:.2f}')

In [ ]:
print(len(G.edges), len(GG.edges))

In [ ]:
errs = []
for i in nodes_from:
    for j in nodes_to:
        errs.append(abs(stat1[i][j]- stat2[i][j])/stat1[i][j])
print(np.mean(errs), np.max(errs))

In [ ]:
cls2hubs = graph_generator.get_cluster_to_bridge_points(G)


In [ ]:
def get_matrix():
    res = {}
    for i in trange(len(communities), position=1):
        res[i] ={}
        f = nx.barycenter(extract_cluster_list_subgraph(G, {i}, communities))[0]
        for j in range(i+1, len(communities)):
            res[i][j] = set()
            points_to = cls2hubs[j]
            for t in points_to:
                p = nx.single_source_dijkstra(G, f, t, weight='length')
                cls = set([G.nodes[u]['cluster'] for u in p[1]])
                for c in cls:
                    res[i][j].add(c)
    return res
A = get_matrix()

In [ ]:
from multiprocessing import Pool

def get_center(data):
    G = data['graph']
    clusters = data['clusters']
    res = {}
    for i in clusters: 
        res[i] = nx.barycenter(graph_generator.extract_cluster_subgraph(G, i))[0]
    return res

def get_path_clusters(data):
    G = data['graph']
    cls2hubs = data['cls2hubs']
    cls2center = data['cls2center']
    communities = data['communities']
    i = data['i']
    res = {i : {}}
    for j in range(i+1, len(communities)):
        res[i][j] = set()
        for to in cls2hubs[j]:
            p = nx.single_source_dijkstra(G, cls2center[i], to, weight='length')
            cls = set([G.nodes[u]['cluster'] for u in p[1]])
            for c in cls:
                res[i][j].add(c)
    return res


def get_aj_matrix(G, communities):
    NUM_THREADS = 6
    cls2hubs = graph_generator.get_cluster_to_bridge_points(G)
    with Pool(NUM_THREADS) as p:
        
        centers_data = [{'graph': G, 'clusters': range(i,len(communities), NUM_THREADS)} for i in range(NUM_THREADS)]
        cls2center = list(tqdm(p.imap_unordered(get_center, centers_data), total = len(centers_data)))
        cls2center = {k: v for d in cls2center for k, v in d.items()}

        centers_data = [{'graph': G, 'cls2hubs': cls2hubs, 'cls2center':cls2center,'communities':communities, 'i':i} for i in range(len(communities))]
        A = list(tqdm(p.imap_unordered(get_path_clusters, centers_data), total = len(centers_data)))
        A = {k: v for d in A for k, v in d.items()} 
    return A
A = get_aj_matrix(G, communities)

In [ ]:
communities = clustering.LouvainKMeansCommunitiesResolver(resolution = 5).find_communities(Q)
clustering.CommunitiesResolver.write_communities(Q, communities)


In [ ]:
import time


data = {}
top_clusters = 4
cls2n = graph_generator.get_cluster_to_neighboring_clusters(Q)
print(cls2n[0])

for i in range(len(communities)):
    data[i] = {}
    for j in cls2n[i]:
        st = list(get_statistic(i,j).items())
        print(st)
        err = 0
        c = set()
        cls = 0
        while err < 0.5:
            err += st[cls][1]
            c1 = set(st[cls][0])
            c = c.union(c1)
            cls+=1
        start = time.time_ns()
        usual_matrix =  get_distance(i,j)
        usual_time = time.time_ns() - start
        start = time.time_ns()
        cluster_matrix =  get_distance(i,j, c)
        cluster_time = time.time_ns() - start
        acc = usual_time / cluster_time
        err = {f : {t : abs(usual_matrix[f][t] - cluster_matrix[f][t])/usual_matrix[f][t] for t in usual_matrix[f]}for f in usual_matrix}
        errs = [err[f][t] for f in err for t in err[f]]
        print(f'from {i} to {j} mean: {np.mean(errs):3f}±{np.std(errs):3f} max: {np.max(errs):3f} acceleration: {acc:3f}')
        # data[i][j] = errs

In [ ]:
def calculate_matrix():
    data = {}
    # cls2n = graph_generator.get_cluster_to_neighboring_clusters(G)
    for i in trange(len(communities), position=0):
        data[i] = {}
        for j in trange(i+1, len(communities)):
            st = list(get_statistic(i,j).items())
            err = 0
            c = set()
            cls = 0
            while err < 0.8:
                err += st[cls][1]
                c1 = set(st[cls][0])
                c = c.union(c1)
                cls+=1
            data[i][j] = c
            if j not in data:
                data[j] = {}
            data[j][i] = c
    return data

In [ ]:
matrix = calculate_matrix()

In [ ]:
N: int = 1000
points = [graph_tests.get_node_for_initial_graph_v2(G) for _ in trange(N, desc='generate points')]

In [ ]:
start = time.time_ns()
for p1, p2 in tqdm(points):
    p = nx.shortest_path(G,p1,p2, weight='length')
usual_time = time.time_ns() - start
usual_time

In [ ]:
start = time.time_ns()
for p1, p2 in tqdm(points):
    c1,c2 = G.nodes[p1]['cluster'], G.nodes[p2]['cluster']
    p = nx.shortest_path(graph_generator.extract_cluster_list_subgraph(G, matrix[c1][c2], communities),p1,p2, weight='length')
usual_time = time.time_ns() - start
usual_time

In [ ]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import pylab


In [ ]:
import math
import os
import pickle
import random
import sys
from multiprocessing import Pool

import networkx as nx
import numpy as np
from scipy.spatial import KDTree as kd


def gen(N):
    W, H = 1000, 1000
    Q = nx.Graph()
    for i in range(N):
        x = random.random() * W
        y = random.random() * H
        Q.add_node(i, x=x, y=y)
    return Q


def add_density(H: nx.Graph, r) -> nx.Graph:
    _G = H.copy()
    ids = [node for node in H.nodes()]
    points = [[d['x'], d['y']] for u, d in H.nodes(data=True)]

    tree = kd(points)
    prob = r - int(r)
    for u, du in H.nodes(data=True):

        dists, n_ids = tree.query([du['x'], du['y']], math.ceil(r))
        if type(n_ids) is np.int64:
            n_ids = [n_ids]
            dists = [dists]
        if math.ceil(r) == 1:
            total = len(n_ids)
        else:
            total = len(n_ids) - 1
            if random.random() < prob:
                total += 1
        for i in range(total):
            _id = n_ids[i]
            d = dists[i]
            if ids[_id] == u:
                continue
            _G.add_edge(u, ids[_id], length=d)
    if not nx.is_connected(_G):
        print('fix connected')
        tmp = []
        for n in nx.connected_components(_G):
            for q in n:
                tmp.append(q)
                break
        for i in range(len(tmp) - 1):
            d1 = _G.nodes[tmp[i]]
            d2 = _G.nodes[tmp[i + 1]]
            _G.add_edge(tmp[i], tmp[i + 1], length=((d1['x'] - d2['x']) ** 2 + (d1['y'] - d2['y']) ** 2) ** 0.5)
    return _G

In [ ]:
Q = add_density(gen(40), 6)
pos= { u : (d['x'],d['y']) for u,d in Q.nodes(data=True)}
# nx.draw_networkx_edge_labels(Q,pos)
# nx.draw(G,pos, node_color =    values, node_size=1500,edge_color=edge_colors,edge_cmap=plt.cm.Reds)
nx.draw(Q,pos, node_size=150,edge_cmap=plt.cm.Reds)
pylab.show()

In [ ]:
def resolve_communities(H: nx.Graph, r: float = 20) -> list[set[int]]:
    communities = nx.community.louvain_communities(H,
                                                   seed=1534,
                                                   weight='length',
                                                   resolution=r)
    cls = []

    for i, c in enumerate(communities):
        for n in nx.connected_components(H.subgraph(c)):
            cls.append(n)
    for i, ids in enumerate(cls):
        for j in ids:
            H.nodes[j]['cluster'] = i
    return cls

In [ ]:
Q = add_density(gen(100), 5)
pos= { u : (d['x'],d['y']) for u,d in Q.nodes(data=True)}
cls = resolve_communities(Q,1
)
print(len(cls))
cls_map = {}
for i in range(len(cls)):
    cls_map[i] = i
values = []
for u,d in Q.nodes(data=True):
    values.append(plt.cm.tab10(cls_map[d['cluster']]))
nx.draw(Q,pos,node_color = values, node_size=150)
pylab.show()